In [8]:
import numpy as np
import pandas as pd
from ULinearModels import ULogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
def getModelError(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    return classification_report(y_test, y_predict)
    
def compareModels(model, uModel, X, y):
    model_error = getModelError(model, X_train, X_test, y_train, y_test)
    uModel_error = getModelError(uModel, X_train, X_test, y_train, y_test)
    print('Ошибка на пакетной модели\n', model_error)
    print('Ошибка на реализованной модели\n', uModel_error)

In [3]:
data_1 = pd.read_csv('DATA/hearing_test.csv')

In [4]:
X = data_1.drop('test_result', axis=1)
y = data_1['test_result']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
model = LogisticRegression()
uModel = ULogisticRegression(alpha=0, gradient_type='default')

In [10]:
compareModels(model, uModel, X, y)

Ошибка на пакетной модели
               precision    recall  f1-score   support

           0       0.93      0.86      0.90       597
           1       0.91      0.96      0.94       903

    accuracy                           0.92      1500
   macro avg       0.92      0.91      0.92      1500
weighted avg       0.92      0.92      0.92      1500

Ошибка на реализованной модели
               precision    recall  f1-score   support

           0       0.93      0.86      0.89       597
           1       0.91      0.95      0.93       903

    accuracy                           0.92      1500
   macro avg       0.92      0.91      0.91      1500
weighted avg       0.92      0.92      0.92      1500

